BaLu_GRAPE: rconv=GCN 
0.0: 0.80 ± 0.03	0.05 ± 0.04	    0.79 ± 0.02	    0.03 ± 0.03	    7.30 ± 3.72	    0.93 ± 0.47	    7.25 ± 1.73	    1.03 ± 0.46
0.1: 1.35 ± 0.25	0.33 ± 0.18	    1.23 ± 0.11	    0.13 ± 0.05	    8.59 ± 5.56	    0.80 ± 0.50	    7.93 ± 2.75	    0.72 ± 0.37
0.3: 2.27 ± 0.71	1.14 ± 0.78	    1.93 ± 0.21	    0.23 ± 0.09	    8.60 ± 5.15	    0.85 ± 0.68	    8.53 ± 1.73     0.71 ± 0.60
BaLu_IGMC:  conv=GCN, rconv=GCN
0.0: 0.80 ± 0.02	0.06 ± 0.03	    0.79 ± 0.02	    0.03 ± 0.02	    7.45 ± 3.96	    0.90 ± 0.61	    7.44 ± 1.76	    1.09 ± 0.38
0.1: 1.31 ± 0.19	0.32 ± 0.20	    1.22 ± 0.12	    0.13 ± 0.07	    8.49 ± 4.63	    0.54 ± 0.34	    7.99 ± 1.64	    0.93 ± 0.80
0.3: 2.17 ± 0.67	1.10 ± 0.81	    1.89 ± 0.18	    0.25 ± 0.18	    10.34 ± 6.24	1.06 ± 0.58	    7.97 ± 0.94     0.64 ± 0.75

# CPU

In [ ]:
import os

cluster_map = {"A100":"""#SBATCH -p kisski
#SBATCH -G A100:1                    
#SBATCH --mem=20G""",
"H100": """#SBATCH -p kisski-h100
#SBATCH -G H100:1                    
#SBATCH --mem=20G""",
"CPU": """#SBATCH --partition=jupyter:cpu
#SBATCH --cpus-per-task=16
#SBATCH --mem=40G"""}

format = """#!/bin/bash
#SBATCH --job-name=BaLu_GNN_{dataset}_{imputer}
{cluster}
#SBATCH --time=12:00:00
#SBATCH --output=run_%x_%j.out
#SBATCH --error=run_%x_%j.err
#SBATCH --mail-type=FAIL # Email on start, end, failure
#SBATCH --mail-user=hao.huang@tib.eu # <-- Replace with your real email
echo "===== JOB STARTED ====="
echo "Hostname: $(hostname)"
echo "Date: $(date)"
echo "User: $USER"
# Load environment
module load miniforge3
module load gcc/13.2.0
module load cuda/11.8
# module load gcc/13.2.0
# module load cuda/12.6.2
# Set up conda
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate /mnt/vast-kisski/projects/kisski-tib-activecl/cenv
# Diagnostics
echo "Which python: $(which python)"
python -c "import torch; print('Torch:', torch.__version__, '| CUDA:', torch.cuda.is_available())"
# Navigate to project directory
cd /mnt/vast-kisski/projects/kisski-tib-activecl/BaLu_Plus/
echo "--- Running: run_exps.py ---"
python3 -u run_exps.py --model_name BaLu_Plus --imputer {imputer} --dataset {dataset} --missing_p 0.0  --gconv {gconv} --rconv {rconv} 
python3 -u run_exps.py --model_name BaLu_Plus --imputer {imputer} --dataset {dataset} --missing_p 0.1  --gconv {gconv} --rconv {rconv} 
python3 -u run_exps.py --model_name BaLu_Plus --imputer {imputer} --dataset {dataset} --missing_p 0.3  --gconv {gconv} --rconv {rconv} 

echo "===== JOB COMPLETED ====="
echo "Date: $(date)"
"""

datasets_org = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']      # network relationships not based on similarity
datasets = [e+"_MCAR" for e in datasets_org]

imputers = ['BaLu_GRAPE', 'BaLu_IGMC', 'GRAPE', 'IGMC']
Layers = [("64 64 64", "64 64"), ("64 64", "64 64"), ("64 64", "64"), ("64 64 64", "64")]
GNNs = ['GCN', 'GAT', 'GraphSAGE', 'RGCN']
rel_dropouts = [0.0, 0.1, 0.2, 0.3]
betas = [0.0, 0.0001, 0.001, 0.01, 0.1]
gammas = [0.0, 0.0001, 0.001, 0.01, 0.1]
etas = [0.0, 0.0001, 0.001, 0.01, 0.1]

default_paras = {'imputer': imputers[0],
                 'dataset': datasets[0], 
                'rel_dropout': rel_dropouts[0],
                'beta': betas[1],
                'gamma': gammas[1],
                'eta': etas[1]}

missing_ps = [0.0, 0.1, 0.3]
print("cd /mnt/vast-kisski/projects/kisski-tib-activecl/BaLu_Plus/")


################################################################################################################################################
cluster = 'CPU'        # H100, CPU
slurm_dir = f'{cluster}_step2_GNN'
################################################################################################################################################

os.makedirs(slurm_dir, exist_ok=True)

def file_name(paras: dict):
    s = ""
    for k, v in paras.items():
        s += f"_{k}={v}"
    return s

commands = []
for dataset in datasets:
    for imputer in imputers[:1]:    # 'BaLu_GRAPE' use only rconv
        for rconv in GNNs:
            gconv = 'GCN'
            slurm_content = format.format(cluster=cluster_map[cluster], dataset=dataset, 
                                          imputer=imputer, gconv=gconv, rconv=rconv)
            filename = os.path.join(slurm_dir, f"{dataset}_imputer={imputer}_gconv={gconv}_rconv={rconv}.slurm")
            with open(filename, "w") as f:
                f.write(slurm_content)
            sbatch_command = f"sbatch {filename}"
            # print(sbatch_command)
            commands.append(sbatch_command)


for dataset in datasets:
    for imputer in imputers[1:2]:    #  'BaLu_IGMC' using both gconv and rconv
        for rconv in GNNs:
            for gconv in GNNs[:3]:
                if rconv != "RGCN":
                    gconv = rconv
                slurm_content = format.format(cluster=cluster_map[cluster], dataset=dataset, 
                                            imputer=imputer, gconv=gconv, rconv=rconv)
                filename = os.path.join(slurm_dir, f"{dataset}_imputer={imputer}_gconv={gconv}_rconv={rconv}.slurm")
                with open(filename, "w") as f:
                    f.write(slurm_content)
                sbatch_command = f"sbatch {filename}"
                # print(sbatch_command)
                commands.append(sbatch_command)

print("\n".join(list(set(commands))))
                

cd /mnt/vast-kisski/projects/kisski-tib-activecl/BaLu_Plus/
sbatch CPU_step2_GNN/Flickr1_M=20_SimRel=0_Rel=1_MCAR_imputer=BaLu_IGMC_gconv=GCN_rconv=GCN.slurm
sbatch CPU_step2_GNN/Syn_M=None_SimRel=1_Rel=4_MCAR_imputer=BaLu_IGMC_gconv=GraphSAGE_rconv=RGCN.slurm
sbatch CPU_step2_GNN/BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR_imputer=BaLu_IGMC_gconv=GAT_rconv=RGCN.slurm
sbatch CPU_step2_GNN/Youtube_M=20_SimRel=1_Rel=4_MCAR_imputer=BaLu_IGMC_gconv=GCN_rconv=GCN.slurm
sbatch CPU_step2_GNN/Syn_M=None_SimRel=1_Rel=4_MCAR_imputer=BaLu_GRAPE_gconv=GCN_rconv=RGCN.slurm
sbatch CPU_step2_GNN/Youtube_M=20_SimRel=1_Rel=4_MCAR_imputer=BaLu_GRAPE_gconv=GCN_rconv=RGCN.slurm
sbatch CPU_step2_GNN/Flickr1_M=20_SimRel=0_Rel=1_MCAR_imputer=BaLu_IGMC_gconv=GraphSAGE_rconv=GraphSAGE.slurm
sbatch CPU_step2_GNN/BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR_imputer=BaLu_IGMC_gconv=GAT_rconv=GAT.slurm
sbatch CPU_step2_GNN/Youtube_M=20_SimRel=1_Rel=4_MCAR_imputer=BaLu_IGMC_gconv=GraphSAGE_rconv=GraphSAGE.slurm
sbatch CPU_step2